## TensorFlow introduction

TensorFlow是Google在2015年11月份Open source 的 deep learning framework（Github项目地址），是之前所開發的深度學習基礎架構DistBelief的強化版，該系統可用於語音識別與影像識別等多個領域。

TensorFlow 的介绍是，一个使用data flow graphs 技術進行數值運算。data flow graphs 中的節點代表數值計算;
节点节点之间的边，代表多维数据(tensors)之间的某种联系。你可以在多种设备（含有CPU或GPU）上通过简单的API调用来使用该系统的功能。TensorFlow是由Google Brain团队的研发人员负责的项目。

## What is Data Flow Graph
数据流图是描述有向图中的数值计算过程。有向图中的节点通常代表数学运算，但也可以表示数据的输入、输出和读写等操作；有向图中的边表示节点之间的某种联系，它负责传输多维数据(Tensors)。图中这些tensors的flow也就是TensorFlow的命名来源。

节点可以被分配到多个计算设备上，可以异步和并行地执行操作。因为是有向图，所以只有等到之前的入度节点们的计算状态完成后，当前节点才能执行操作。

## TensorFlow Feature
1. 灵活性  
TensorFlow不是一个严格的神经网络工具包，只要你可以使用数据流图来描述你的计算过程，你可以使用TensorFlow做任何事情。你还可以方便地根据需要来构建数据流图，用简单的Python语言来实现高层次的功能。
2. 可移植性  
TensorFlow可以在任意具备CPU或者GPU的设备上运行，你可以专注于实现你的想法，而不用去考虑硬件环境问题，你甚至可以利用Docker技术来实现相关的云服务。
3. 提高开发效率  
TensorFlow可以提升你所研究的东西产品化的效率，并且可以方便与同行们共享代码。
4. 支持语言选项  
目前TensorFlow支持Python和C++语言。（但是你可以自己编写喜爱语言的SWIG接口）
5. 充分利用硬件资源，最大化计算性能

## It is how to work with TensorFlow：
* 将计算流程表示成图；
* 通过Sessions来执行图计算；
* 将数据表示为tensors；
* 使用Variables来保持状态信息；
* 分别使用feeds和fetches来填充数据和抓取任意的操作结果；


http://www.jeyzhang.com/tensorflow-learning-notes.html
https://www.tensorflow.org/tutorials/

## Build Graph
在TensorFlow中，Constant是一种没有输入的ops，但是你可以将它作为其他ops的输入。Python库中的ops构造器将返回构造器的输出。TensorFlow的Python库中有一个默认的图，将ops构造器作为节点，更多可了解Graph Class文档。

In [17]:
import tensorflow as tf

# Create a Constant op that produces a 1x2 matrix.  The op is
# added as a node to the default graph.
#
# The value returned by the constructor represents the output
# of the Constant op.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix
# multiplication.
product = tf.matmul(matrix1, matrix2)

默认的图(Default Graph)现在有了三个节点：两个 Constant()ops和一个matmul()op。为了得到这两个矩阵的乘积结果，还需要在一个session中启动图计算。

### 在Session中执行图计算

Sessions 最后需要关闭，以释放相关的资源；你也可以使用with模块，session在with模块中自动会关闭：

In [18]:
# Launch the default graph.
sess = tf.Session()

# To run the matmul op we call the session 'run()' method, passing 'product'
# which represents the output of the matmul op.  This indicates to the call
# that we want to get the output of the matmul op back.
#
# All inputs needed by the op are run automatically by the session.  They
# typically are run in parallel.
#
# The call 'run(product)' thus causes the execution of threes ops in the
# graph: the two constants and matmul.
#
# The output of the op is returned in 'result' as a numpy `ndarray` object.
result = sess.run(product)
print(result)
# ==> [[ 12.]]

# Close the Session when we're done.
sess.close()

[[ 12.]]


In [19]:
with tf.Session() as sess:
    result = sess.run([product])
    print(result)

[array([[ 12.]], dtype=float32)]


TensorFlow的这些节点最终将在计算设备(CPUs,GPus)上执行运算。如果是使用GPU，默认会在第一块GPU上执行，如果你想在第二块多余的GPU上执行：  

with tf.Session() as sess:  
    with tf.device("/gpu:1"):  
        matrix1 = tf.constant([[3., 3.]])  
        matrix2 = tf.constant([[2.],[2.]])  
        product = tf.matmul(matrix1, matrix2)

## 交互环境下的使用
以上的python示例中，使用了Session和Session.run()来执行图计算。然而，在一些Python的交互环境下(如IPython中)，你可以使用InteractiveSession类，以及Tensor.eval()、Operation.run()等方法。例如，在交互的Python环境下执行以下代码：

In [20]:
# Enter an interactive TensorFlow Session.
import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# Initialize 'x' using the run() method of its initializer op.
x.initializer.run()

# Add an op to subtract 'a' from 'x'.  Run it and print the result
sub = tf.subtract(x, a)
print(sub.eval())
# ==> [-2. -1.]

# Close the Session when we're done.
sess.close()

[-2. -1.]


## Tensors
TensorFlow中使用tensor数据结构（实际上就是一个多维数据）表示所有的数据，并在图计算中的节点之间传递数据。一个tensor具有固定的类型、级别和大小，更加深入理解这些概念可参考Rank, Shape, and Type。

## 变量(Variables)
变量在图执行的过程中，保持着自己的状态信息。下面代码中的变量充当了一个简单的计数器角色：

赋值函数assign()和add()函数类似，直到session的run()之后才会执行操作。与之类似的，一般我们会将神经网络模型中的参数表示为一系列的变量，在模型的训练过程中对变量进行更新操作。

In [21]:
# Create a Variable, that will be initialized to the scalar value 0.
state = tf.Variable(0, name="counter")

# Create an Op to add one to `state`.

one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# Variables must be initialized by running an `init` Op after having
# launched the graph.  We first have to add the `init` Op to the graph.
init_op = tf.initialize_all_variables()

# Launch the graph and run the ops.
with tf.Session() as sess:
  # Run the 'init' op
  sess.run(init_op)
  # Print the initial value of 'state'
  print(sess.run(state))
  # Run the op that updates 'state' and print 'state'.
  for _ in range(3):
    sess.run(update)
    print(sess.run(state))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0
1
2
3


## 抓取(Fetches)
为了抓取ops的输出，需要先执行session的run函数。然后，通过print函数打印状态信息。  
所有tensors的输出都是一次性 [连贯] 执行的。

In [22]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
  result = sess.run([mul, intermed])
  print(result)

# output:
# [array([ 21.], dtype=float32), array([ 7.], dtype=float32)]

[21.0, 7.0]


## 填充(Feeds)
TensorFlow也提供这样的机制：先创建特定数据类型的占位符(placeholder)，之后再进行数据的填充。例如下面的程序：

In [23]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))

[array([ 14.], dtype=float32)]


## 曲线拟合
下面是一段使用Python写的，曲线拟合计算。官网将此作为刚开始介绍的示例程序。

In [24]:
# 简化调用库名
import tensorflow as tf
import numpy as np

# 模拟生成100对数据对, 对应的函数为y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype("float32")
y_data = x_data * 0.1 + 0.3

# 指定w和b变量的取值范围（注意我们要利用TensorFlow来得到w和b的值）
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

# 最小化均方误差
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# 初始化TensorFlow参数
init = tf.initialize_all_variables()

# 运行数据流图（注意在这一步才开始执行计算过程）
sess = tf.Session()
sess.run(init)

# 观察多次迭代计算时，w和b的拟合值
for step in xrange(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
(0, array([-0.24438691], dtype=float32), array([ 0.67934817], dtype=float32))
(20, array([-0.02117219], dtype=float32), array([ 0.36550206], dtype=float32))
(40, array([ 0.06546878], dtype=float32), array([ 0.31866655], dtype=float32))
(60, array([ 0.09015942], dtype=float32), array([ 0.30531955], dtype=float32))
(80, array([ 0.09719568], dtype=float32), array([ 0.30151594], dtype=float32))
(100, array([ 0.09920083], dtype=float32), array([ 0.30043203], dtype=float32))
(120, array([ 0.09977224], dtype=float32), array([ 0.30012313], dtype=float32))
(140, array([ 0.09993511], dtype=float32), array([ 0.30003509], dtype=float32))
(160, array([ 0.09998152], dtype=float32), array([ 0.30001], dtype=float32))
(180, array([ 0.09999473], dtype=float32), array([ 0.30000284], dtype=float32))
(200, array([ 0.0999985], dtype=float32), array([ 0.30000082], dtype=float32))


## Build CNN model

first you need to load dataset.  
there use sample dataset at tensorflow

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


import librarys

In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

define method to use
1. Variable init
2. Convolution
3. Pooling  
padding='SAME'表示通过填充0，使得输入和输出的形状一致。

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### paras layer

In [5]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

### conv layer-1

In [6]:
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### conv layer-2

In [7]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### full connection

In [8]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### drop-out
use drop out tech to avoid over-fitting

In [9]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### output layer-softmax

In [10]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
y_ = tf.placeholder(tf.float32, [None, 10])

### model training

In [11]:
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.arg_max(y_conv, 1), tf.arg_max(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### train 

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())


for i in range(200):
    batch = mnist.train.next_batch(50)

    if i % 20 == 0:
        train_accuacy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuacy))
    train_step.run(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 0.5})

# accuacy on test
sess.close()
print("test accuracy %g"%(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy 0.1
step 20, training accuracy 0.46
step 40, training accuracy 0.52
step 60, training accuracy 0.58
step 80, training accuracy 0.82
step 100, training accuracy 0.84
step 120, training accuracy 0.9
step 140, training accuracy 0.92
step 160, training accuracy 0.9
step 180, training accuracy 0.92
